# Final Project: San Francisco Fire Department Service Calls

In [1]:
import datetime as dt
import os
import pandas as pd
from sodapy import Socrata

In [18]:
def api_query(SoQL_query):
    """
    https://dev.socrata.com/foundry/data.sfgov.org/enhu-st7v
    The function is created from code given to query data off the
    API of the government website. Personal login information
    is used to access the API. Additionally, a SoQL query is
    used as an input to determine how the data should be accessed.
    
    Args: SoQL_query, a SQL-like query using SocrataQL
    Return: queried dataset in dataframe format
    """
    client = Socrata(domain='data.sfgov.org', 
                     app_token='LatHs7KifDEpxpxnlKMb9SFfy', 
                     username="qzyu999@gmail.com", 
                     password="SFData999")

    result_list = client.get("enhu-st7v", limit=5000000, where=SoQL_query)

    return(pd.DataFrame.from_records(result_list))

In [19]:
def clean_my_data(df, col_list):
    """
    Takes in the dataframe that needs to be cleaned. Also takes in the list of column names of
    complicated and long dates, for proper date conversion.
    
    Args: 'df' (the dataframe to be cleaned)
          'col_list' (the list of complicated datetime column names)
    Return: 'df' (the dataframe already cleaned)
    """
    # Simple data type conversions to int
    df["number_of_alarms"] = df["number_of_alarms"].astype(int)
    df["unit_sequence_in_call_dispatch"] = df["unit_sequence_in_call_dispatch"].astype(int)
    
    # Simple datetime conversions
    df["call_date"] = pd.to_datetime(df["call_date"], format="%Y-%m-%d")
    df["watch_date"] = pd.to_datetime(df["watch_date"], format="%Y-%m-%d")
    
    # For every value in the provided complicated datetime column list, change data types to datetimes
    for val in col_list:
        df[val] = pd.to_datetime(df[val], format="%Y-%m-%dT%H:%M:%S.%f")
        
    # Day of the week columns
    # Source: https://stackoverflow.com/questions/25146121/extracting-just-month-and-year-from-pandas-datetime-column-python
    df["year"] = pd.DatetimeIndex(df["call_date"]).year
    df["month"] = pd.DatetimeIndex(df["call_date"]).month
    df["dotw"] = df["call_date"].dt.day_name()
        
    return df

In [27]:
def get_clean_query(SoQL_query_timeframe,
                    date_and_time_col_list,
                    lodate=0,
                    hidate=0):
    """
    Queries data from API of the San Francisco government website, using a specified timeframe.
    Cleans the data, and returns the clean dataframe. Also returns a date-subsetted dataframe
    depending on user choice.
    
    Args: 'SoQL_query_timeframe' (a SQL-like query using SocrataQL),
          'date_and_time_col_list' (the list of complicated datetime column names),
          'lodate' (optional field, subsets by lowest date)
          'hidate' (optional field, subsets by highest date)
    Returns:
    """
    # Query according the timeframe that user specifies
    df_query = api_query(SoQL_query_timeframe)
    # Clean all datatypes, add columns, etc.
    clean_df = clean_my_data(df=df_query, col_list=date_and_time_col_list)
    
    # Subset df by date if desired
    if lodate != 0:
        date_subset_df = clean_df[clean_df["watch_date"] >= lodate]
    if hidate !=0:
        date_subset_df = clena_df[clean_df["watch_date"] <= hidate]
    if (lodate == 0) & (hidate == 0):
        date_subset_df = "Not Specified"
    
    # Return the original df (all dates) and the date subsetted df
    return clean_df, date_subset_df

In [28]:
post_2015 = 'watch_date>="2015-01-01T00:00:00"'
date_column_list = ["received_dttm", "entry_dttm", "dispatch_dttm", "response_dttm",
                    "on_scene_dttm", "transport_dttm", "hospital_dttm", "available_dttm"]
low_date = "2015-01-01"
high_date = "2018-11-20"

In [29]:
df_2015, df1 = get_clean_query(SoQL_query_timeframe=post_2015,
                               date_and_time_col_list=date_column_list,
                               lodate=low_date,
                               hidate=high_date)

In [30]:
df1.dtypes

address                                      object
als_unit                                       bool
available_dttm                       datetime64[ns]
battalion                                    object
box                                          object
call_date                            datetime64[ns]
call_final_disposition                       object
call_number                                  object
call_type                                    object
call_type_group                              object
city                                         object
dispatch_dttm                        datetime64[ns]
entry_dttm                           datetime64[ns]
final_priority                               object
fire_prevention_district                     object
hospital_dttm                        datetime64[ns]
incident_number                              object
location                                     object
neighborhoods_analysis_boundaries            object
number_of_al

In [31]:
# This list makes a lot of sense, not every incident requires hospital, transportation, etc.
# Unfortunately we are missing a lot of on-scene date times though?
df1.isna().sum().sort_values(ascending=False)

hospital_dttm                        861678
transport_dttm                       854221
on_scene_dttm                        237770
response_dttm                         32222
city                                   2924
station_area                           2213
zipcode_of_incident                    1532
call_type_group                         848
available_dttm                          251
original_priority                        99
box                                      53
final_priority                            0
entry_dttm                                0
dispatch_dttm                             0
dotw                                      0
call_type                                 0
fire_prevention_district                  0
call_final_disposition                    0
call_date                                 0
battalion                                 0
als_unit                                  0
call_number                               0
neighborhoods_analysis_boundarie

In [32]:
df1["watch_date"].max()

Timestamp('2018-11-23 00:00:00')